In [ ]:



import os
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

mapping_dict = np.load("dataset/mapping_fileB1+B2.npy", allow_pickle=True).item()
print(mapping_dict)
feature_data = []  # Feature list
label_list = []    # Firmness list
plot_oi=[]

keys = list(mapping_dict.keys())
print(f"Keys from mapping_dict: {keys[:5]}")  # Debug: Print first 5 keys

label_list=list(mapping_dict.values())
print(label_list)


{'RS20240205125747_P19100983.xlsx': 32.2687, 'RS20240205125849_P19100983.xlsx': 32.2687, 'RS20240205130001_P19100983.xlsx': 32.2687, 'RS20240205130054_P19100983.xlsx': 32.2687, 'RS20240205130211_P19100983.xlsx': 32.2687, 'RS20240205130308_P19100983.xlsx': 32.2687, 'RS20240205130405_P19100983.xlsx': 32.2687, 'RS20240205130458_P19100983.xlsx': 32.2687, 'RS20240205130603_P19100983.xlsx': 32.2687, 'RS20240205130708_P19100983.xlsx': 32.2687, 'RS20240205130805_P19100983.xlsx': 32.2687, 'RS20240205130910_P19100983.xlsx': 32.2687, 'RS20240205131007_P19100983.xlsx': 32.2687, 'RS20240205131102_P19100983.xlsx': 32.2687, 'RS20240205131155_P19100983.xlsx': 32.2687, 'RS20240205131248_P19100983.xlsx': 32.2687, 'RS20240205131338_P19100983.xlsx': 32.2687, 'RS20240205131439_P19100983.xlsx': 32.2687, 'RS20240205131541_P19100983.xlsx': 32.2687, 'RS20240205131639_P19100983.xlsx': 32.2687, 'RS20240205131820_P19100983.xlsx': 39.0408, 'RS20240205131914_P19100983.xlsx': 39.0408, 'RS20240205132004_P19100983.xls

In [2]:
csv_dir = "dataset/data/B1+B2xlsx/"
if not os.path.exists(csv_dir):
    raise FileNotFoundError(f"Directory {csv_dir} does not exist.")
z=0
for filename in os.listdir(csv_dir):
    file_key = os.path.splitext(filename)[0]
    if file_key + ".xlsx" in keys:  # Ensure extension matches keys
        file_path = os.path.join(csv_dir, filename)
        #print(file_path)
        df = pd.read_excel(file_path,header=None)  # Use read_excel for .xlsx files
        features = df.iloc[:, 2].values  # Ensure column index is correct
        # features variable has all the values of the 3rd column i.e y
        #z+=1
        plot_ing = df.iloc[:,1]
        #print(len(features),z)#prints 512 2773 times
        feature_data.append(features)
        plot_oi.append(plot_ing)
        #print(feature_data[0])
        #feature data is an arrray where every element is the features i.e the 3rd row of every file
columns = [f"Column {i+1}" for i in range(512)]
df_x = pd.DataFrame(feature_data, columns=columns)
df_plot = pd.DataFrame(plot_oi)
#print(df_x.head())#prints [5 rows x 512 columns] values

In [3]:
df_y = pd.DataFrame(label_list, columns=['Firmness'])
#print(df_y.head()) 2773
#print(len(df_y))#prints

In [4]:
from sklearn.model_selection import train_test_split



from sklearn.metrics import mean_squared_error,r2_score
x_train,x_test,y_train,y_test=train_test_split(df_x,df_y,test_size=0.2,random_state=19)




In [5]:
# performing preprocessing part
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()

X_train = sc.fit_transform(x_train)
X_test = sc.transform(x_test)

In [6]:
# Applying PCA function on training
# and testing set of X component
from sklearn.decomposition import PCA

pca = PCA(n_components=2)

X_train = pca.fit_transform(X_train)
X_test = pca.transform(X_test)

explained_variance = pca.explained_variance_ratio_

In [7]:
print(X_train)

[[ 4.92726519  3.90275954]
 [29.11081554  7.98445878]
 [-8.61317576  8.69787711]
 ...
 [-9.00525989 -0.1187129 ]
 [-4.60893479 -2.04321519]
 [-5.76958537  1.07548791]]


In [14]:
from sklearn.linear_model import LinearRegression

linear_regression = LinearRegression()
linear_regression.fit(X_train, y_train)
y_pred_lr = linear_regression.predict(X_test)

In [15]:
print("The r2 error for PCA Followed by linear Regression :",r2_score(y_pred_lr,y_test))
print("The mean squared error for PCA Followed by linear Regression :",mean_squared_error(y_pred_lr,y_test))

The r2 error for PCA Followed by linear Regression : -167.7901642674375
The mean squared error for PCA Followed by linear Regression : 458.8399496093646


In [8]:
from sklearn.ensemble import RandomForestRegressor
rfr = RandomForestRegressor(random_state=13)
rfr.fit(X_train,y_train)
y_pred = rfr.predict(X_test)

c:\Users\Hyper\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\base.py:1152: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


In [9]:
from sklearn.metrics import mean_squared_error,r2_score
mean_squared_error(y_pred,y_test)


506.8028606623103

In [10]:
r2_score(y_pred,y_test)

-5.902388467179013